In [2]:
import numpy as np
import pandas as pd

In [3]:
ind_movies= pd.read_csv('indian movies.csv',usecols=['ID','Movie Name','Year','Genre','Language'])

In [4]:
ind_movies.head()

,ID,Movie Name,Year,Genre,Language
0,tt0398974,Dr. Shaitan,1960,-,hindi
1,tt1702558,Nadir Khan,1968,-,urdu
2,tt0493437,Apna Sapna Money Money,2006,"Comedy, Musical, Romance",hindi
3,tt0273405,Aag Aur Sholay,1987,-,urdu
4,tt0049595,Parivar,1956,"Comedy, Drama, Family",hindi


In [5]:
ind_movies['Language'].value_counts()

Language
hindi        14350
malayalam     6200
tamil         6000
telugu        5300
bengali       4200
urdu          3447
kannada       3233
marathi       2300
punjabi       1800
nepali        1000
oriya          850
gujarati       700
bhojpuri       500
assamese       450
rajastani      100
sanskrit        48
kashmiri        43
konkani         43
tulu            38
Name: count, dtype: int64

In [6]:
ind_movies.isna().sum()

ID               0
Movie Name       0
Year          1561
Genre            0
Language         0
dtype: int64

In [7]:
ind_movies.replace('-',np.nan,inplace=True)

In [8]:
ind_movies.isna().sum()

ID            2508
Movie Name       0
Year          1561
Genre         8581
Language         0
dtype: int64

In [9]:
ind_movies.shape

(50602, 5)

In [10]:
def random_impute(dataset,column):
    sample= dataset[column].dropna().sample(dataset[column].isna().sum())
    missing_values=dataset[dataset[column].isna()]
    sample.index=missing_values.index
    dataset.loc[dataset[column].isna(),column]=sample
    

In [11]:
random_impute(ind_movies,'Genre')

In [12]:
ind_movies.isna().sum()

ID            2508
Movie Name       0
Year          1561
Genre            0
Language         0
dtype: int64

In [13]:
ind_movies['Year'].value_counts()

Year
2018               1934
2019               1892
2017               1678
2016               1509
2015               1339
                   ... 
2002 TV Special       1
2021 TV Special       1
XVII 2016             1
2015–2017             1
VII 2021              1
Name: count, Length: 579, dtype: int64

In [14]:
def clean_year(year):
    print(year)
    if (re.sub('[^0-9]',"",year))[:4]:
        return int((re.sub('[^0-9]',"",year))[:4])
    else:
        return 1960
        
    
    
    

In [15]:
ind_movies=ind_movies.ffill()

In [16]:
ind_movies.isna().sum()

ID            0
Movie Name    0
Year          0
Genre         0
Language      0
dtype: int64

In [18]:
import re

In [19]:
ind_movies['Year']=ind_movies['Year'].apply(clean_year)

1960
1968
2006
1987
1956
2008
2019
2007
1939
1992
1997
2018 Video
2016
1976
1958
1986
2013
2017 Video
1988
2012
2017
1974
1991
1990
2020
1988
2009
1978
1999
1935
1970
1985
1981
2000
2000
1987
2003
1967
1933
1996
2010
II 2017
II 2017
1995
1973
2000
2015
1950
2019
2015
I 2008
I 2008
1966
2021
1994
1989
2016
1997
2009
1938
1969
1984
1952
1987
2015
II 1976
2013
2016
1997
2018
1996
2009
1977
1957
1966
2018
I 2014
1988
1990
2016
1937
2013
2005
1988
1988
2014
2018
2009
1989
1950
1988
I 2016
2000
2021
2014
1982
2003
1999
1987
1982
2021
2007
1992
1976
1983
2019 Video
2016
2021 TV Movie
1981
2003
1997
1985
1984
2008
2013 TV Movie
2007
2014
1989
1965
1981
2019
1996
1970
II
2009
2004
2018
1959
2006
1996
2017
2012
2011
2012
2011
1999
2003
2020
2020– 
1994
1961
2012
I 2010
1999
2009
2009
1986
2015
1978
1977
1971
1962
2018
2010
1968
2003
1977
1982
2002
1940
2002–2003
2020
1988
2018
1950
2018– 
2008
2015
1995
2018
2011
1991
2011
2015
2022
1976
1986
2017
I 2020
2009
2018 TV Movie
2015
2020
1984
1978
19

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
tf_g= TfidfVectorizer()
tf_m= TfidfVectorizer()
tf_l= TfidfVectorizer()
genre_num_matrix=tf_g.fit_transform(ind_movies['Genre'])
movie_num_matrix=tf_m.fit_transform(ind_movies['Movie Name'])
lang_num_matrix=tf_l.fit_transform(ind_movies['Language'])

                              

In [23]:
year_cs=cosine_similarity([[1995]],np.array(ind_movies['Year']).reshape(-1,1)).flatten()

In [100]:
def search(Movie_Name):
    all_records=ind_movies[ind_movies['Movie Name'].str.lower()== Movie_Name.lower()][['Genre','Year','Language']].sort_values('Year',ascending=False).head()
    regex_1 = re.compile(r'\b' + re.escape(Movie_Name) + r'\w*', re.IGNORECASE)
    regex_2=re.compile(f'.*{re.escape(Movie_Name)}.*',re.IGNORECASE)
    starts_with_records=ind_movies[ind_movies['Movie Name'].str.contains(regex_1,na=False)].index
    contains_records=ind_movies[ind_movies['Movie Name'].str.contains(regex_2,na=False)].index
    if len(all_records)==0:
        print('going inside regex')
        #regex_1= re.compile(f'{re.escape(Movie_Name)}.*',re.IGNORECASE)
        if len(starts_with_records)>0:
            return ind_movies.iloc[starts_with_records]
        elif len(contains_records)>0:
            return ind_movies.iloc[contains_records]
        else:
            return 'No movies found based on the search'
    elif len(all_records)>1:
        records=ind_movies[ind_movies['Movie Name'].str.lower()== Movie_Name.lower()][['Genre','Year','Language']].sort_values('Year',ascending=False).head(1)
        multiple_records=ind_movies[ind_movies['Movie Name'].str.lower()== Movie_Name.lower()][['Genre','Year','Language']].sort_values('Year',ascending=False).head().index
        next_records=multiple_records[1:]
        similar_names=ind_movies.iloc[next_records]
        len_sim_nm=len(similar_names)
        #multiple_rec=ind_movies[ind_movies['Movie Name']== Movie_Name].head()[['Genre','Year','Language']].sort_values('Year',ascending=False).index
    else:
        records=ind_movies[ind_movies['Movie Name'].str.lower()== Movie_Name.lower()][['Genre','Year','Language']].sort_values('Year',ascending=False).head(1)
        len_sim_nm=0
                      
    genre= records['Genre'].iloc[0]
    if ',' in genre:
        genre_list=genre.split(',')
        genre_text=''.join(genre_list)
    else:
        genre_text=genre
    year=records['Year'].iloc[0]
    language=records['Language'].iloc[0]
    genre=tf_g.transform([genre_text])
    movie=tf_m.transform([Movie_Name])
    language=tf_l.transform([language])
    genre_cs=cosine_similarity(genre,genre_num_matrix).flatten()
    #print(genre_cs)
    
    #print(top_5_indices)
    movie_cs=cosine_similarity(movie,movie_num_matrix).flatten()
    lang_cs=cosine_similarity(language,lang_num_matrix).flatten()
    year_cs=cosine_similarity([[year]],np.array(ind_movies['Year']).reshape(-1,1)).flatten()
    combined_cs=genre_cs+movie_cs+lang_cs+year_cs
    #print(movie_cs)
    top_5_indices=np.argsort(combined_cs)[-5:][::-1]
    if len_sim_nm>0:
        if len(starts_with_records)>0:
            del_sim_from_reg= starts_with_records[len_sim_nm:]
            regex_records= ind_movies.iloc[del_sim_from_reg]
            return pd.concat([ind_movies.iloc[top_5_indices],regex_records,similar_names])
            
        #return pd.concat([ind_movies.iloc[top_5_indices],similar_names])
    else:
        del_sim_from_reg= starts_with_records[1:]
        regex_records= ind_movies.iloc[del_sim_from_reg]
        return pd.concat([ind_movies.iloc[top_5_indices],regex_records])
    #print(genre,year)
    
    
    
    

In [84]:
regex_1= re.compile(f'{re.escape("ma")}.*',re.IGNORECASE)
k=ind_movies[ind_movies['Movie Name'].str.contains(regex_1,na=False)].index

In [85]:
ind_movies.iloc[k]

,ID,Movie Name,Year,Genre,Language
8,tt0154875,Midnight Mail,1939,Action,hindi
11,tt8272296,A Mayalu Timi Lai,2018,"Short, Music",nepali
15,tt0318983,Amman Koil Kizhakkaalae,1986,Drama,tamil
17,tt7836398,Kamal Khatri Feat. Simpal Kharel: Jaula Relaima,2017,"Short, Music",nepali
20,tt6517874,Ramante Edenthottam,2017,"Comedy, Drama, Romance",malayalam
...,...,...,...,...,...
50583,tt2971414,Prema Katha Chitram,2013,"Drama, Horror",telugu
50585,tt0329964,Aalu Magalu,1957,Drama,telugu
50590,tt0390208,Manchi Rojulu Vastai,1972,"Musical, Drama",telugu
50594,tt0073191,Jai Santoshi Maa,1975,"Drama, Fantasy",hindi


In [129]:
ind_movies[(ind_movies['Language']=='telugu') & ind_movies['Year'].isin([2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024])]

,ID,Movie Name,Year,Genre,Language
86,tt9053354,Anthervedam,2018,Thriller,telugu
126,tt7757972,Saakshyam,2018,Action,telugu
173,tt4384242,Pataas,2015,"Action, Comedy",telugu
236,tt11536276,3 Monkeys,2020,Drama,telugu
284,tt8178538,Vasudhaika 1957,2016,Horror,telugu
...,...,...,...,...,...
50336,tt8333978,Maharshi,2019,"Action, Drama",telugu
50375,tt7322890,MLA,2018,"Action, Drama",telugu
50378,tt5597792,Attack,2016,"Action, Crime, Thriller",telugu
50586,tt9277948,Akshara,2021,Thriller,telugu


In [50]:
records=ind_movies[ind_movies['Movie Name']== "Anthervedam"][['Genre','Year','Language']].sort_values('Year',ascending=False).head(1)
multiple_records=ind_movies[ind_movies['Movie Name']== "Anthervedam"][['Genre','Year','Language']].sort_values('Year',ascending=False).head().index
next_records=multiple_records[1:]
similar_names=ind_movies.iloc[next_records]
print(len(similar_names))

0


In [107]:
search('pawan')

,ID,Movie Name,Year,Genre,Language
48674,tt3805166,Pawan,2014,"Documentary, Short, Family",nepali
6607,tt5994520,68 Members,2014,"Documentary, Short, Family",nepali
5507,tt0927739,Punam,2006,"Documentary, Short, Family",nepali
215,tt5994746,Supportive Men,2014,"Documentary, Short, Family",nepali
33145,tt9536822,Kusunda,2021,"Documentary, Short",nepali
13504,tt5321634,Pawan Basanti,1980,Drama,hindi
18687,tt0358008,Patit Pawan,1955,Drama,hindi
21986,tt11157870,Pawan Raja,2017,Drama,bhojpuri
32837,tt0242768,Pawan Putra Hanuman,1957,Fantasy,hindi
34156,tt0236581,Patit Pawan,1933,Drama,hindi


You can search whatever you want in above search function. I am trying to use widget to enter the input.

In [87]:
import ipywidgets as widgets
from IPython.display import display

In [89]:
input_movie_name= widgets.Text(value="Nadir Khan", description="Search for the movie",disabled=False)

movies_output= widgets.Output()

In [90]:
def on_input(text):
    with movies_output:
        movies_output.clear_output()
        output=search(text['new'])
        if len(output)==5:
            display(output)
        else:
            display("Movie name not found")

input_movie_name.observe(on_input, names="value")

display(input_movie_name,movies_output)

Text(value='Nadir Khan', description='Search for the movie')

Output()